<a href="https://colab.research.google.com/github/SlavikHom/Learning/blob/main/Text_generation_on_%22War_and_peace%22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
import numpy as np
import os
import time


In [ ]:
path_to_file = "/content/pg2600.txt"

In [ ]:
text = open(path_to_file,'rb').read().decode(encoding='utf-8')
print(len(text))
print(text[:300])

3293519
﻿The Project Gutenberg eBook of War and Peace, by Leo Tolstoy

This eBook is for the use of anyone anywhere in the United States and
most other parts of the world at no cost and with almost no restrictions
whatsoever. You may copy it, give it away or re-use it under the terms
of the Project Gut


In [ ]:
vocab = sorted(set(text))
print(len(vocab))
vocab[0:40]

113


['\n',
 '\r',
 ' ',
 '!',
 '"',
 '#',
 '$',
 '%',
 "'",
 '(',
 ')',
 '*',
 ',',
 '-',
 '.',
 '/',
 '0',
 '1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 ':',
 ';',
 '=',
 '?',
 'A',
 'B',
 'C',
 'D',
 'E',
 'F',
 'G',
 'H',
 'I',
 'J']

In [ ]:
ids_from_chars = tf.keras.layers.StringLookup(vocabulary=list(vocab))
ids_from_chars

In [ ]:
tf.strings.unicode_split(text,"UTF-8")

<tf.Tensor: shape=(3293519,), dtype=string, numpy=
array([b'\xef\xbb\xbf', b'T', b'h', ..., b'\n', b'\r', b'\n'],
      dtype=object)>

In [ ]:
all_ids = ids_from_chars(tf.strings.unicode_split(text,"UTF-8"))
all_ids

<tf.Tensor: shape=(3293519,), dtype=int64, numpy=array([113,  50,  66, ...,   1,   2,   1])>

In [ ]:
ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids)
ids_dataset

<_TensorSliceDataset element_spec=TensorSpec(shape=(), dtype=tf.int64, name=None)>

In [ ]:
chars_from_ids = tf.keras.layers.StringLookup(vocabulary=ids_from_chars.get_vocabulary(), invert = True)
chars_from_ids

In [ ]:
for ids in ids_dataset.take(10):
  print(ids, chars_from_ids(ids).numpy().decode('utf-8'))

tf.Tensor(113, shape=(), dtype=int64) ﻿
tf.Tensor(50, shape=(), dtype=int64) T
tf.Tensor(66, shape=(), dtype=int64) h
tf.Tensor(63, shape=(), dtype=int64) e
tf.Tensor(3, shape=(), dtype=int64)  
tf.Tensor(46, shape=(), dtype=int64) P
tf.Tensor(76, shape=(), dtype=int64) r
tf.Tensor(73, shape=(), dtype=int64) o
tf.Tensor(68, shape=(), dtype=int64) j
tf.Tensor(63, shape=(), dtype=int64) e


In [ ]:
seq_length = 100
sequences = ids_dataset.batch(seq_length+1,drop_remainder=True)
for seq in sequences.take(1):
  print(chars_from_ids(seq))


tf.Tensor(
[b'\xef\xbb\xbf' b'T' b'h' b'e' b' ' b'P' b'r' b'o' b'j' b'e' b'c' b't'
 b' ' b'G' b'u' b't' b'e' b'n' b'b' b'e' b'r' b'g' b' ' b'e' b'B' b'o'
 b'o' b'k' b' ' b'o' b'f' b' ' b'W' b'a' b'r' b' ' b'a' b'n' b'd' b' '
 b'P' b'e' b'a' b'c' b'e' b',' b' ' b'b' b'y' b' ' b'L' b'e' b'o' b' '
 b'T' b'o' b'l' b's' b't' b'o' b'y' b'\r' b'\n' b'\r' b'\n' b'T' b'h' b'i'
 b's' b' ' b'e' b'B' b'o' b'o' b'k' b' ' b'i' b's' b' ' b'f' b'o' b'r'
 b' ' b't' b'h' b'e' b' ' b'u' b's' b'e' b' ' b'o' b'f' b' ' b'a' b'n'
 b'y' b'o' b'n' b'e' b' '], shape=(101,), dtype=string)


In [ ]:
def split_input_target(sequence):
  input_text = sequence[:-1] 
  target_text = sequence[1:]
  return input_text, target_text

split_input_target(list("Mister Fox"))

(['M', 'i', 's', 't', 'e', 'r', ' ', 'F', 'o'],
 ['i', 's', 't', 'e', 'r', ' ', 'F', 'o', 'x'])

In [ ]:
dataset = sequences.map(split_input_target)


In [ ]:
BATCH_SIZE = 64
BUFFER_SIZE = 10000
dataset = (dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True).prefetch(tf.data.experimental.AUTOTUNE))
dataset

<_PrefetchDataset element_spec=(TensorSpec(shape=(64, 100), dtype=tf.int64, name=None), TensorSpec(shape=(64, 100), dtype=tf.int64, name=None))>

In [ ]:
vocab_size = len(ids_from_chars.get_vocabulary())
embedding_dim = 256
rnn_units = 1024
class MyModel(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, rnn_units):
    super().__init__(self)
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(rnn_units, return_sequences=True, return_state=True)
    self.dense = tf.keras.layers.Dense(vocab_size)
  def call(self, x, states = None, training = False, return_state = False):
    x = self.embedding(x, training = training)
    if states is None:
      states = self.gru.get_initial_state(x)
    x, states = self.gru(x, initial_state=states, training=training)
    x = self.dense(x, training=training)
    if return_state:
      return x, states
    else:
      return x
  
model = MyModel(vocab_size = vocab_size, embedding_dim = embedding_dim, rnn_units=rnn_units)


In [ ]:
for input_example_batch, target_example_batch in dataset.take(1):
  example_batch_predictions = model(input_example_batch)
  print(example_batch_predictions.shape)

(64, 100, 114)


In [ ]:
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)
model.compile(optimizer='adam',loss=loss)
model.summary()

Model: "my_model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     multiple                  29184     
                                                                 
 gru_3 (GRU)                 multiple                  3938304   
                                                                 
 dense_3 (Dense)             multiple                  116850    
                                                                 
Total params: 4,084,338
Trainable params: 4,084,338
Non-trainable params: 0
_________________________________________________________________


In [ ]:
epochs = 200
history = model.fit(dataset, epochs=epochs)

In [ ]:
import pickle
file_path = '/content/history.pkl'
with open(file_path,'wb') as file:
  pickle.dump(history, file)

In [ ]:
with open(file_path, 'rb') as file:
  loaded = pickle.load(file)

print(loaded.history)

{'loss': [2.1212563514709473, 1.3937631845474243, 1.239816665649414, 1.1697149276733398, 1.1228948831558228, 1.0862354040145874, 1.0543851852416992, 1.0260474681854248, 0.9992814660072327, 0.9739484190940857, 0.9510160088539124, 0.9293086528778076, 0.9097944498062134, 0.8929044008255005, 0.8785170316696167, 0.8661101460456848, 0.8554233908653259, 0.8477506637573242, 0.8412443399429321, 0.8376451134681702, 0.8341829776763916, 0.8342028260231018, 0.8357345461845398, 0.8368052840232849, 0.8364286422729492, 0.8476063013076782, 0.851637065410614, 0.8588782548904419, 0.8755120038986206, 0.8633057475090027]}


Такс

In [ ]:
class OneStep(tf.keras.Model):
  def __init__(self, model, chars_from_ids, ids_from_chars, temperature=1.0):
    super().__init__()
    self.temperature = temperature
    self.model = model
    self.chars_from_ids = chars_from_ids
    self.ids_from_chars = ids_from_chars
  
    # 
    skip_ids = self.ids_from_chars(['UNK'])[:, None]
    sparse_mask = tf.SparseTensor(
        values = [-float('inf')]*len(skip_ids),
        indices=skip_ids,
        dense_shape=[len(ids_from_chars.get_vocabulary())]
    )
    self.prediction_mask = tf.sparse.to_dense(sparse_mask)

  @tf.function
  def generate_one_step(self,inputs,states=None):
    # превращаем строки в айдишники
    input_chars = tf.strings.unicode_split(inputs,"UTF-8")
    input_ids = self.ids_from_chars(input_chars).to_tensor()

    # запускаем модель, которую обучали выше 
    predicted_logits, states = self.model(x=input_ids, states=states, return_state=True)

    predicted_logits = predicted_logits[:,-1,:]
    predicted_logits = predicted_logits / self.temperature

    # применяем UNK вместо неизвестных символов если они есть
    predicted_logits = predicted_logits + self.prediction_mask

    # генерируем символ
    predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
    predicted_ids = tf.squeeze(predicted_ids, axis=-1)

    predicted_chars = self.chars_from_ids(predicted_ids)

    return predicted_chars, states

one_step_model = OneStep(model, chars_from_ids, ids_from_chars)

In [ ]:
states = None
next_char = tf.constant(["I am Groot"])
result = [next_char]
for n in range(1000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
print(result[0].numpy().decode('utf-8'))

I am Groot, having risen he stood at turn room, suddenly
eight-yet in her: as a battle, he except that,” he said, gentleman
army, saw teard horsemen and thought Rostóv, when he conare.

It homes age, she ceased
a Sternemple Prince Andrew spoke of, more intrusing stay, the
very girl. And you have forgotten there was, my dear, suddenly remarked
is doiniters?” thought Prince Andrew went is soon sat, quickly) “I love him with him. He murded
poor suggested him quarrel. Yet would go away. In. We must surprise, but on the cuay
capture of the Highness.”

Parist—formed again and presented a dreadow man to may his head in his arm, as
the what though sleppisned his face was saying and whispered:

“I have tired along—the Emps. Why does not a going up to him that needed when
it is very quiet to everything is, much. That’s not at
a march, dear? I remark. “And what is in love,” and sat? He
was a clarm sewilliant, a loud across the sunting as on the French
drunker cross
onto the chief dinner,
